<a href="https://colab.research.google.com/github/MateusGheorghe/LAMP/blob/master/Step_2_Parameter_Tuning_and_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
# TensorFlow e tf.keras
import tensorflow as tf

In [0]:
# Librariesauxiliares
import itertools
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
import pandas as pd
import time
sns.set()
import plotly.express as px
from sklearn import preprocessing
#Model Selection
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_validate, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn import metrics
import pickle
#MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [0]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
import matplotlib.pyplot as plt
SMALL_SIZE = 22
MEDIUM_SIZE = 42
BIGGER_SIZE = 62
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [52]:
print('The tensorflow version is {}.'.format(tf.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The tensorflow version is 2.1.0.
The scikit-learn version is 0.22.1.


In [0]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print ('\nElapsed time is ')
        print (str(time.time() - startTime_for_tictoc))
        print('seconds.\n')
    else:
        print ('\nToc: start time not set\n')

In [0]:
def TIC():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc1
    startTime_for_tictoc1 = time.time()

def TOC():
    import time
    if 'startTime_for_tictoc' in globals():
        print ('\nElapsed time is ')
        print (str(time.time() - startTime_for_tictoc1))
        print('seconds.\n')
    else:
        print ('\nToc: start time not set\n')

In [0]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [0]:
TIC()
T0 = time.time()

* POSITION = 0 -> r0 / POSITION = 1-> r1
* N_CLAS -> Number of classes in the dataset loaded (Unitl April 4th: ND, CH, SC, CR, and CQ)
* seed -> seed used in the hyperparameter tuning and model selection
* SAVE-> 0 do not save / 1 do save

In [0]:
POSITION=0 
N_CLAS=5
seed = 8
SAVE = 1

In [0]:
if POSITION==0:
  with open('Dataset_r0.pickle', 'rb') as filename:
    Dataset = pickle.load(filename)
elif POSITION==1:
  with open('Dataset_r1.pickle', 'rb') as filename:
    Dataset = pickle.load(filename)

In [59]:
Dataset

,PC0,PC1,PC2,PC3,Label,Class,Severity,Severity Norm,Reg Division Cat,Reg Division Num
0,-0.462714,0.467353,1.179220,-1.147022,1,CH,0.200000,0.243243,low,0
1,-0.473318,0.445630,1.166120,-1.105849,1,CH,0.205000,0.249324,low,0
2,-0.483762,0.423800,1.152240,-1.064692,1,CH,0.210000,0.255405,low,0
3,-0.493919,0.401869,1.137749,-1.023868,1,CH,0.215000,0.261486,low,0
4,-0.486881,0.421357,1.133990,-1.059043,1,CH,0.220000,0.267568,low,0
...,...,...,...,...,...,...,...,...,...,...
441,1.940860,-1.829271,2.009975,1.002267,0,ND,0.099444,0.120946,low,0
442,1.939265,-1.842192,2.047630,1.019873,4,CQ,0.105278,0.128041,low,0
443,1.937643,-1.854460,2.084513,1.037541,4,CQ,0.111111,0.135135,low,0
444,1.935907,-1.866127,2.120780,1.054449,4,CQ,0.116944,0.142230,low,0


In [60]:
X=np.array(Dataset.values[:,0:np.size(Dataset,axis=1)-6],dtype='float64')
y=np.array(Dataset['Label'],dtype='int')
INPUT_DIM=np.size(X,axis=1)#Number of features
print('Number of features:{}'.format(INPUT_DIM))

Number of features:4


# Hyperparameter Tuning / Model Selection

Function to run randomized search CV in diverse models with repeated stratified K-fold

In [0]:
# adapted from: http://www.codiply.com/blog/hyperparameter-grid-search-across-multiple-models-in-scikit-learn/
#Single Metric
# class EstimatorSelectionHelper:
    
#     def __init__(self, models, params):
#         self.models = models
#         self.params = params
#         self.keys = models.keys()
#         self.grid_searches = {}
    
#     def fit(self, X, y, CV=5, n_exec=1, n_repet=2, seed=1,**grid_kwargs):
#         for key in self.keys:
#             print('Running Randomized Search CV for %s.' % key)
#             model = self.models[key]
#             params = self.params[key]
#             skf=RepeatedStratifiedKFold(n_splits=CV,n_repeats=n_repet,random_state=seed)
#             grid_search = RandomizedSearchCV(model, params, cv=skf, n_iter=n_exec, **grid_kwargs)
#             grid_search.fit(X, y)
#             self.grid_searches[key] = grid_search
#         print('Done.')
    
#     def score_summary(self, sort_by='mean_test_score'):
#         frames = []
#         best_params=[]
#         best_score=[]
#         best_data=[]
#         for name, grid_search in self.grid_searches.items():
#             frame = pd.DataFrame(grid_search.cv_results_)
#             frame = frame.filter(regex='^(?!.*param_).*$')
#             frame['estimator'] = len(frame)*[name]
#             frames.append(frame)
#             bs=(name,grid_search.best_score_)
#             bp=(name,grid_search.best_params_)
#             bdata=(name,grid_search.cv_results_)
#             best_params.append(bp)
#             best_score.append(bs)
#             best_data.append(bdata)
#         df = pd.concat(frames)
        
#         df = df.sort_values([sort_by], ascending=False)
#         df = df.reset_index()
#         df = df.drop(['rank_test_score', 'index'], 1)
        
#         columns = df.columns.tolist()
#         columns.remove('estimator')
#         columns = ['estimator']+columns
#         df = df[columns]
#         return df,best_params,best_score,best_data

In [0]:
# adapted from: http://www.codiply.com/blog/hyperparameter-grid-search-across-multiple-models-in-scikit-learn/
#MULTI METRIC
class EstimatorSelectionHelper:
    
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, CV=5, n_exec=1, n_repet=2, seed=1,**grid_kwargs):
        for key in self.keys:
            t1 = time.time()
            print('Running Randomized Search CV for %s.' % key)
            model = self.models[key]
            params = self.params[key]
            scoring = {'Accuracy':'accuracy','AUC': 'roc_auc_ovr_weighted'}
            skf=RepeatedStratifiedKFold(n_splits=CV,n_repeats=n_repet,random_state=seed)
            grid_search = RandomizedSearchCV(model, params, cv=skf, n_iter=n_exec,scoring=scoring,refit='Accuracy',random_state=seed,**grid_kwargs)
            grid_search.fit(X, y)
            self.grid_searches[key] = grid_search
            print ('Elapsed time '+str(round(time.time()-t1,4))+' seconds')
        print('Done.')
    
    def score_summary(self, sort_by='mean_test_Accuracy'):
        frames = []
        best_params=[]
        best_score=[]
        best_data=[]
        for name, grid_search in self.grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
            bs=(name,grid_search.best_score_)
            bp=(name,grid_search.best_params_)
            bdata=(name,grid_search.cv_results_)
            best_params.append(bp)
            best_score.append(bs)
            best_data.append(bdata)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_Accuracy','rank_test_AUC', 'index'], 1)
        N=int((np.size(df,1)-10)/2)
        for i in range(0,N):
          df = df.drop(['split'+str(i)+'_test_Accuracy'], 1)
          df = df.drop(['split'+str(i)+'_test_AUC'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df,best_params,best_score,best_data

In [0]:
def NNet(n_neurons=10,n_hidden_layers=1,t_activation='relu'):
# create model
  if n_hidden_layers==1:
    model = tf.keras.Sequential([tf.keras.layers.Dense(n_neurons,input_dim=INPUT_DIM, activation=t_activation),tf.keras.layers.Dense(N_CLAS, activation='softmax')]) #5 classes
  elif n_hidden_layers==2:
    model = tf.keras.Sequential([tf.keras.layers.Dense(n_neurons,input_dim=INPUT_DIM ,activation=t_activation),tf.keras.layers.Dense(n_neurons, activation=t_activation),tf.keras.layers.Dense(N_CLAS, activation='softmax')]) #5 classes
  elif n_hidden_layers==3:
    model = tf.keras.Sequential([tf.keras.layers.Dense(n_neurons,input_dim=INPUT_DIM, activation=t_activation),tf.keras.layers.Dense(n_neurons, activation=t_activation),tf.keras.layers.Dense(n_neurons, activation=t_activation),tf.keras.layers.Dense(N_CLAS, activation='softmax')]) #5 classes
  # Compile model
  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])	
  return model

In [0]:
MLPClassifier= tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=NNet,verbose=0)

Representants related to top 10 of paper: 

In [0]:
modelsHT = {
            'SVM': SVC(C=1,kernel='rbf',degree=5,gamma='scale',tol=1e-3,class_weight='balanced',decision_function_shape='ovr',random_state=seed,
                       max_iter=1000000,probability=True),
            'RF':RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=None,
                                                min_samples_split=2,min_samples_leaf=1,min_weight_fraction_leaf=0.0,
                                                max_features=None, bootstrap=True,oob_score=False, random_state=seed, 
                                                class_weight='balanced'),
            'ABST':AdaBoostClassifier(base_estimator=None,n_estimators=50,learning_rate=1.0,algorithm='SAMME.R',random_state=seed),
            'GBST':GradientBoostingClassifier(learning_rate=0.1,n_estimators=100,max_depth=3,
                                                min_samples_split=2,min_samples_leaf=1,min_weight_fraction_leaf=0.0,random_state=seed,
                                                max_features=None,tol=1e-4),
            'BAG':BaggingClassifier(base_estimator=None,n_estimators=10,bootstrap=True,random_state=seed),
            'MLP':MLPClassifier}



*   SVM: Based on paper -> *Effectiveness of Random Search in SVM hyper-parameter tuning*

*   RF: Based on paper -> *Hyperparameters and tuning strategies for random forest* -- https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

* ABST: https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/

* GBST: https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/

* BAG: Based on previous variations


* MLP: Based on preliminary grid-search


In [0]:
paramsHT = { 
            'SVM': {'C':list(np.logspace(-2, 15, 18)),'kernel':['rbf','poly','linear','sigmoid'],'degree':[2,3,4,5],'gamma':[1e-15,1e-14,1e-13,1e-12,1e-11,1e-10,1e-09,1e-08,1e-07,1e-06,1e-05,0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0,'scale','auto']},
            'RF':{'n_estimators':[10,50,100,200,500,1000],'criterion':['gini','entropy'],'max_features':list(np.arange(1,np.size(X,axis=1)+1)),'min_samples_split':list(np.arange(2,11)),'bootstrap':[True,False],'min_samples_leaf':list(np.arange(2,11))},
            'ABST':{'n_estimators':[10,50,100,200,500,1000],'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0, 5, 10]},
            'GBST':{'n_estimators':[10,50,100,200,500,1000],'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 1.0, 5, 10],
            'min_samples_split':[2, 5, 10],'min_samples_leaf':[1, 2, 4],'max_features':list(np.arange(1,np.size(X,axis=1)+1))},
            'BAG':{'n_estimators':[10,50,100],'bootstrap':[True,False],'max_features':list(np.arange(1,np.size(X,axis=1)+1))},
            'MLP':{'n_neurons':[5,10,20,50,100],'n_hidden_layers':[1,2,3],'t_activation':['relu','tanh'],'epochs':[150,200,250,300,350],'batch_size':[2,4,6,8,10,20,30,32]}
            }

In [0]:
tic()
Kf=10
Exec=50
Rep=10
helper1 = EstimatorSelectionHelper(modelsHT, paramsHT)
helper1.fit(X, y, CV=Kf, n_exec=Exec, n_repet=Rep, seed=seed,n_jobs=1)
toc()

Running Randomized Search CV for RF.
Elapsed time 3174.6285 seconds
Running Randomized Search CV for MLP.


In [0]:
df,best_params,best_scores,best_data=helper1.score_summary()

In [0]:
df

In [0]:
best_params

In [0]:
best_scores

In [0]:
best_data

In [0]:
timestr = time.strftime("%Y_%m_%d-%H_%M_%S")
TF=round(time.time()-T0,4)
FILE_NAME=('Models_Kfold_'+str(Kf)+'_Combinations_'+str(Exec)+'_Repetitions_'+str(Rep)+'_date_'+timestr+'.pickle')
if SAVE==1:
  with open(FILE_NAME, 'wb') as filename:
    pickle.dump([TF,Kf,Exec,Rep,df,best_params,best_scores,best_data], filename)  

In [0]:
#PARA CARREGAR DADOS POSTERIORMENTE
# with open('Models_Kfold_10_Combinations_1_Repetitions_1_date_2020_04_08-19_05_08.pickle', 'rb') as filename:
#   Tempo,X1,X2,X3,X4,X5,X6,X7 = pickle.load(filename)

In [0]:
TOC()